In [1]:
from dataclasses import dataclass, field
from typing import List, Literal
from datetime import datetime, timedelta

from database import SchedulerStorage
from model import Slot, Event, Scheduler

import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination



In [ ]:
storage = SchedulerStorage();

class SchedulerProblem(ElementwiseProblem):
    def __init__(self, slots: List[Slot], events: List[Event]):
        self.slots = slots;
        self.events = events;
        
        # One variables per event - > the 0 to 1 scale of event.min to event.max time
        n_var = len(events);
        
        n_constr = 0;
        
        # Decision variable bounds (0 to 1)
        xl = np.zeros(n_var);
        xu = np.ones(n_var);

        super().__init__(n_var=n_var, n_obj=2, n_constr=n_constr, xl=xl, xu=xu);

    # ================== ASSUMPTIONS ============================================ #
    # Slots are given in the order that they should be used
    # Events are also given in the order that they need to be scheduled
    #    - > assume scheduling in order, who cares about creating a custom order

    def _evaluate(self, x, out, *args, **kwargs):
        total_slot_priority = 0;
        num_late = 0;
        
        slot_index = 0;
        out_of_space = False;

        # walk through each event, with each x
        for x_index, event in enumerate(self.events):
            
            
            # first check to see if current slot has room for event
            if self.slots[slot_index].capacity < event.calc_duration(x[x_index]):
                # move to next slot if previous too small
                slot_index += 1;
                
                # check to see if out of bounds
                if slot_index > len(self.slots): out_of_space = True;
                    
            
            # assign event to slot
            if not out_of_space:
                self.slots[slot_index] += event.calc_duration(x[x_index])

                
            
            # Calculate objectives
            total_slot_priority += slot.priority
            
            if slot.start + event.min_time > event.due_date:
                num_late += 1

        out["F"] = [total_slot_priority, num_late]
        
        


In [5]:
scheduler = Scheduler(storage)
slots = scheduler.slots
events = scheduler.events

In [9]:
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling

problem = SchedulerProblem(slots, events)

algorithm = NSGA2(pop_size=20,
            sampling=IntegerRandomSampling(),
            crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
            mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
            eliminate_duplicates=True,
            )

termination = get_termination("n_gen", 15)

result = minimize(
    problem,
    algorithm,
    termination,
    verbose=False
)

print("Optimization Completed!")
print("Best Solutions:")
for i, solution in enumerate(result.X):
    print(solution, end=' - ')
    print(result.F[i])
    
    

Optimization Completed!
Best Solutions:
[0 1 1 0] - [4. 0.]
[1 0 1 0] - [4. 0.]
[1 1 1 0] - [4. 0.]
[1 0 0 1] - [4. 0.]
[1 1 1 1] - [4. 0.]
[0 0 1 0] - [4. 0.]
[0 1 0 0] - [4. 0.]
[1 0 1 1] - [4. 0.]
[1 1 0 1] - [4. 0.]
[0 0 0 1] - [4. 0.]
[0 0 1 1] - [4. 0.]
[0 0 0 0] - [4. 0.]
[0 1 1 1] - [4. 0.]
[0 1 0 1] - [4. 0.]
[1 1 0 0] - [4. 0.]
[1 0 0 0] - [4. 0.]
